In [1]:
from torchvision import transforms
import  torch.nn as nn
import torch
import torch.nn.functional as F

In [2]:
_tasks = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
    ])

In [3]:

from torchvision.datasets import MNIST

In [4]:
## Load MNIST Dataset and apply transformations
mnist = MNIST("data", download=True, train=True, transform=_tasks)

In [5]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [6]:
## create training and validation split 
split = int(0.8 * len(mnist))
index_list = list(range(len(mnist)))
train_idx, valid_idx = index_list[:split], index_list[split:]

In [7]:
## create sampler objects using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

In [8]:
## create iterator objects for train and valid datasets
trainloader = DataLoader(mnist, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(mnist, batch_size=256, sampler=val_sampler)

In [9]:
import torch.nn.functional as F

In [10]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(784, 128)
        self.output = nn.Linear(128, 10)
  
    def forward(self, x):
        x = self.hidden(x)
        x = F.sigmoid(x)
        x = self.output(x)
        return x

In [11]:
model = Model().cuda()

In [12]:
from torch import optim
import numpy as np

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [13]:
for epoch in range(1, 11): ## run the model for 10 epochs
    train_loss, valid_loss = [], []
    
    ## training part 
    model.train()
    for data, target in trainloader:
        optimizer.zero_grad()
        data = data.view(data.size(0), -1)
        data=data.to('cuda:0')
        target=target.to('cuda:0')
                ## 1. forward propagation
        output = model(data)
        #print(type(target))
        ## 2. loss calculation
        loss = F.cross_entropy(output, target)
        
        ## 3. backward propagation
        loss.backward()
        
        ## 4. weight optimization
        optimizer.step()
        
        train_loss.append(loss.item())
            ## evaluation part 
    model.eval()
    for data, target in validloader:
        data = data.view(data.size(0), -1)
        data = data.to('cuda:0')
        target = target.to('cuda:0')
        output = model(data)
        loss = loss_function(output, target)
        valid_loss.append(loss.item())

    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))

    

/home/aditya/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1 Training Loss:  0.5366171749823905 Valid Loss:  0.2807141786560099
Epoch: 2 Training Loss:  0.25771358046442905 Valid Loss:  0.22702250962561749
Epoch: 3 Training Loss:  0.21885145657715646 Valid Loss:  0.2040645958895379
Epoch: 4 Training Loss:  0.20034513467962436 Valid Loss:  0.2015788047871691
Epoch: 5 Training Loss:  0.1792773344018992 Valid Loss:  0.18156144815556546
Epoch: 6 Training Loss:  0.17879866234007033 Valid Loss:  0.22072622528735628
Epoch: 7 Training Loss:  0.17563688493472465 Valid Loss:  0.17195965191151233
Epoch: 8 Training Loss:  0.16561831676579536 Valid Loss:  0.17638572733452978
Epoch: 9 Training Loss:  0.1569447063226649 Valid Loss:  0.18537676381938
Epoch: 10 Training Loss:  0.1650906631604154 Valid Loss:  0.2083394226241619


In [15]:
## dataloader for validation dataset 
dataiter = iter(validloader)

data, labels = dataiter.next()
data=data.to('cuda:0')
data = data.view(data.size(0), -1)
output = model(data)

/home/aditya/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [17]:
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor)

In [19]:
print ("Actual   :", labels[:10])
print ("Predicted:", preds[:10])

Actual   : tensor([9, 2, 3, 0, 4, 4, 1, 7, 9, 4])
Predicted: tensor([9, 2, 3, 0, 4, 4, 1, 7, 9, 4], device='cuda:0')
